<a href="https://colab.research.google.com/github/aquapathos/FooocusJAni/blob/main/Fooocus_onColabwithMM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](https://i.imgur.com/WaGbhNe.png)

# Foooocus + MM LoRA

lllyasvielさんの作られた [Fooocus Fooocus](https://github.com/lllyasviel/Fooocus) に

- 日本語メニューのリソース
- 画家　[渡辺美香子さんの色鉛筆画](https://www.facebook.com/mikakostudio/)風LoRA　のプリセット  （絵の例は[こちら](https://wgarts.stores.jp/)）
- [Fooocus_Extensions_AlekPet](https://github.com/AlekPet/Fooocus_Extensions_AlekPet)

を加えて起動できるようにした Google Colaboratory のノート（pythonプログラム)を作りました。  
日本語化しての起動については、[Fooocus 日本語アニメ特化版 j.aicu.ai/FoooC](https://j.aicu.ai/FoooC)を参考にさせてもらっています。
さらに、[Fooocus_Extensions_AlekPet](https://github.com/AlekPet/)のプロンプト翻訳機能も使えるようにしてあります。


## 変更点
1. pygit2==1.12.2　→1.15.1
2. アニメ用のモデルを、animaPencilXL_v310.safetensors から　animagine-xl-3.0.safetensors　に変更
3. MM(Mikako Method)のプリセットを追加
4. 日本語化リソースの強化

# １．画像出力フォルダの設定

- 出力画像をGoogle Driveに残すことができます。  
残したい場合は下のセルで「Google Driveに保存」にチェックを入れ、パスをセットしてください。
> - /content/gdrive/MyDrive/がGoogle Drive 直下を意味します。  
> - ローカルなストレージ（例えばCドライブ）は指定できません。  
> - 出力画像をGoogle ドライブに残す場合は、アクセスを許可を求めるダイアログが開きますので、許可し「Googleドライブに接続」を完了してください。
- Google Driveに残さない場合でも、セッションが生きている間は画像を参照可能です。「高度な設定」にチェックを入れ、Setting タブの一番下のHistory Logからアクセスできます。

### 参考
- 生成画像は１枚２MB前後、約500枚で１GB
- Google ドライブの無料ユーザーの容量は15 GB


### <font color=red>↓実行してください</font>


In [ ]:
# @markdown #保存フォルダの選択
from google.colab import drive
import os

# オプションの設定

Googleドライブに保存 = False  # @param {type:"boolean"}

if Googleドライブに保存:
  # Google Driveをマウント
  drive.mount('/content/gdrive')
  保存フォルダ = "/content/gdrive/MyDrive/Fooocus-outputs"  # @param {type:"string"}

target_local_dir = "/content/Fooocus/outputs"

%cd /content
try:
  setup_finished
except NameError:
  # 必要なパッケージのインストール
  !pip -qq install pygit2==1.15.1
  !rm -fr Fooocus
  !git clone https://github.com/lllyasviel/Fooocus.git

  if Googleドライブに保存:
    # シンボリックリンクの作成
    if os.path.exists(保存フォルダ ):
      if os.path.islink(保存フォルダ ):
        os.unlink(保存フォルダ )
      os.symlink(保存フォルダ,target_local_dir)
    else:
      # Google 出力ディレクトリを作成
      os.makedirs(target_local_dir, exist_ok=True)

# 日本語パッチを追加
ja_json_url = "https://raw.githubusercontent.com/aquapathos/FooocusJAni/refs/heads/main/ja.json"
ja_json_path = "/content/Fooocus/language/ja.json"
os.makedirs(os.path.dirname(ja_json_path), exist_ok=True)
!curl -L -o {ja_json_path} {ja_json_url}
print(f"ja.json downloaded and saved to {ja_json_path}")

# MM用のプリセットを追加
mm_json_url = "https://huggingface.co/iciromaco/mm/raw/main/mm_preset.json"
mm_json_path = "/content/Fooocus/presets/mm_preset.json"
os.makedirs(os.path.dirname(mm_json_path), exist_ok=True)
!curl -L -o {mm_json_path} {mm_json_url}
print(f"mm_preset.json downloaded and saved to {mm_json_path}")

# アニメ版の設定ファイルのパッチ(最新版)を追加
anime_json_url = "https://raw.githubusercontent.com/aquapathos/FooocusJAni/refs/heads/main/anime.json"
anime_json_path = "/content/Fooocus/presets/anime.json"
os.makedirs(os.path.dirname(anime_json_path), exist_ok=True)
!curl -L -o {anime_json_path} {anime_json_url}
print(f"anime.json downloaded and saved to {anime_json_path}")

setup_finished = True

## ２．プロンプト翻訳機能の導入（オプション、試験的導入）
次のセルを実行することで、プロンプトの翻訳機能が使えるようになります。   
デフォルトはオフです（英語で入力しなければいけない）

試したい場合は次のセルを実行し、ウェブアプリ上で　**Prompt Translate**　にチェックを入れてください。さらに、**See translated prompts after click Generate**にチェックを入れることでどんな英語に翻訳されたのかを知ることもできます。

これは試験導入ですのでうまく動作しないようなら最初に戻って再実行し、次のセルは飛ばしてください。

In [ ]:
!git clone https://github.com/AlekPet/Fooocus_Extensions_AlekPet.git
!mv /content/Fooocus_Extensions_AlekPet/prompt_translate/webui.py /content/Fooocus
!mv /content/Fooocus_Extensions_AlekPet/prompt_translate/modules/* /content/Fooocus/modules


## ３．ウェブアプリの起動

#### モデルタイプ： 次の３つから選んでください
- デフォルト : Fooocus のデフォルト設定
- アニメ：　アニメ画生成
- MM：Mikako Methodの風景画生成

モデルのダウンロードに少し時間がかかります。しばらくお待ちください。

### モデルタイプを選び<font color=red>↓実行してください</font>

In [ ]:
%cd /content/Fooocus

# オプションの設定
日本語の使用 = True  # @param {type:"boolean"}
モデルタイプ = "MM"  # @param ["デフォルト", "アニメ","MM"] {allow-input: true}

# Fooocus JP Script Options
if モデルタイプ == "デフォルト" and not 日本語の使用:
    !python entry_with_update.py  --always-high-vram  --output-path {target_local_dir} --share
elif モデルタイプ == "デフォルト" and 日本語の使用 :
    !python entry_with_update.py  --always-high-vram  --output-path {target_local_dir} --share --language ja
elif モデルタイプ == "アニメ" and not 日本語の使用:
    !python entry_with_update.py --preset anime --always-high-vram --output-path {target_local_dir} --share
elif モデルタイプ == "アニメ" and 日本語の使用:
    !python entry_with_update.py --preset anime --always-high-vram --output-path {target_local_dir} --share --language ja
elif モデルタイプ == "MM" and not 日本語の使用:
    !python entry_with_update.py --preset mm_preset --always-high-vram --output-path {target_local_dir} --share
elif モデルタイプ == "MM" and 日本語の使用:
    !python entry_with_update.py --preset mm_preset --always-high-vram --output-path {target_local_dir} --share --language ja

次のように、**gradio.live** で終わるURLが表示されたらクリックしてください。約５分かかります。

<img src="https://i.imgur.com/YARA2nf.png" width = 500></img>

ブラウザでアプリが立ち上がったあと、アプリ上で**enable translate**にチェックをいれると日本語でプロンプトを入力することができます。

<img src="https://i.imgur.com/TmGn36m.png" width = 500>></img>



|||
| ---- | ---- |
|![](https://i.imgur.com/jaQBei0.png)|![](https://i.imgur.com/XwvS5Zx.png)|

----
>#### 左
>upside-down landscape reflected in lake, no human, a small country house stands, a red roadster in front of the house, a colorful cosmos field in the foreground

>#### 右
>flower, outdoors, sky, tree, no humans, night, traditional media, grass, star sky, night sky, scenery, starry sky, mountain, painting medium, field, house


